<a href="https://colab.research.google.com/github/icarito/UPSK-SQL001-SQLZoo-murder/blob/sql-murder/Sql-Murder-Mystery/SQL_Murder_Mystery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Murder Mystery!
Adapted By: Martin Arroyo

![Detective making connections between points](https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/sleuth.png?raw=1)

**Credit**

This material was adapted from the [SQL Murder Mystery by Knight Lab](https://mystery.knightlab.com/) under [Creative Commons CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/). The SQL Murder Mystery was originally created by [Joon Park](https://twitter.com/joonparkmusic) and [Cathy He](https://twitter.com/Cathy_MeiyingHe) while they were Knight Lab fellows. See the [GitHub repository](https://github.com/NUKnightLab/sql-mysteries) for more information.

## Escenario

¡Se ha cometido un crimen y los detectives necesitan tu ayuda! Te dieron el informe de la escena del crimen, pero de alguna manera lo perdiste. Recuerdas vagamente que el crimen fue un `murder` _(asesinato)_ que ocurrió en algún momento el 15 de Enero de 2018 y que tuvo lugar en `SQL City`. Depende de ti averiguar quién es el asesino usando solo tus habilidades en SQL y tu ingenio. Te proporcionan una conexión a la base de datos del Departamento de Policía, que tiene todas las pistas que necesitarás para atrapar al asesino.

Usa las habilidades que desarrollaste en SQL 101 y 102, junto con cualquier recurso que desees, para resolver el `SQL Murder Mystery`!"

## Conéctate a la base de datos del Departamento de Policía

Para comenzar y ejecutar tus consultas, presiona play en la celda de abajo para conectarte a la base de datos del Departamento de Policía.

Para ejecutar consultas, crea una nueva celda de `Code` y escribe `%%sql` en la parte superior. Luego puedes escribir tus consultas debajo. Ve el ejemplo a continuación:
```python
%%sql

SELECT *
FROM table

```

In [1]:
%%capture
# @title Press Play { display-mode: "form" }
# Install `teachdb` and `coop_grader`
print("Installing `teachdb` and its dependencies...")
!pip install --quiet --upgrade git+https://github.com/freestackinitiative/teachingdb.git git+https://github.com/martinmarroyo/coop_grader.git
print("Successfully installed `teachdb`")
import pandas as pd
from teachdb.teachdb import connect_teachdb
from coop_grader.sql_murder_mystery.check_suspect import check_suspect
# Set configurations for notebook
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 99)
# Load data
con = connect_teachdb(database="sql_murder_mystery")

%sql con

## Descubriendo tablas en la base de datos

Comenzamos nuestra búsqueda para encontrar al asesino explorando la base de datos del Departamento de Policía. Pero aún no has visto la base de datos y no sabes cuáles son las tablas, así que, ¿cómo sabes qué buscar?

Afortunadamente, la mayoría de los sistemas de gestión de bases de datos relacionales tienen esta información almacenada en un lugar donde puedes consultarla. Muy a menudo, se utiliza un esquema especial conocido como [`information_schema`](https://en.wikipedia.org/wiki/Information_schema) para almacenar información sobre las tablas y columnas en tu base de datos (también conocido como metadata). La base de datos del Departamento de Policía tiene un information schema, con la vista `tables` que te muestra qué tablas están disponibles, y la vista `columns` que te muestra todas las columnas de cada tabla y sus tipos de datos.

### Listando todas las tablas en la base de datos de la Policía

#### Revisa la [descripción de `information_schema.tables`](https://duckdb.org/docs/sql/information_schema.html#tables-and-views)

Primero, veremos todas las tablas disponibles para nosotros consultando la vista `information_schema.tables`. Te daremos la primera consulta para empezar, pero de aquí en adelante tendrás que idear las consultas restantes utilizando tu conocimiento de SQL y tu ingenio.

Aquí está la consulta necesaria para mostrarte las tablas en la base de datos del Departamento de Policía. Cópiala/Pégala en la celda de abajo y ejecútala para ver las tablas disponibles para ti:

>```sql
>SELECT *
>FROM information_schema.tables
>```

In [ ]:
%%sql
SELECT *
FROM information_schema.tables

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,memory,main,crime_scene_report,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,memory,main,drivers_license,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,memory,main,facebook_event_checkin,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,memory,main,get_fit_now_check_in,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,memory,main,get_fit_now_member,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,memory,main,income,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,memory,main,interview,BASE TABLE,None,None,None,None,None,YES,NO,None,None
7,memory,main,person,BASE TABLE,None,None,None,None,None,YES,NO,None,None


### Listando todas las tablas y sus columnas en la base de datos de la Policía

**Revisa la [descripción de `information_schema.columns`](https://duckdb.org/docs/sql/information_schema.html#columns)**

¡Genial! Ahora que sabes qué tablas están disponibles, es hora de averiguar las columnas que tiene cada tabla junto con el tipo de datos de cada columna. Escribe una consulta que muestre el nombre de la tabla, el nombre de la columna y el tipo de datos para cada tabla en la base de datos del Departamento de Policía utilizando la vista `information_schema.columns`. Asegúrate de que la salida esté ordenada por nombre de tabla y luego por nombre de columna (ascendente):

*Pistas opcionales - ¡intenta usarlas solo si te quedas **REALMENTE** atascado!*

<details>
<summary>Pista 1</summary>
<p>Estructura tu consulta de la misma manera que lo hicimos en la consulta anterior donde miramos `information_schema.tables`</p>
</details>

<details>
<summary>Pista 2</summary>
<p>Asegúrate de revisar el enlace para la descripción de information_schema.columns. Te dirá los nombres de las columnas que debes usar para la consulta. Recuerda, queremos el nombre de la tabla, el nombre de la columna y el tipo de datos - ¡revisa la descripción para encontrar los nombres apropiados!</p>
</details>

<details>
<summary>Pista 3</summary>
<p>No olvides usar la declaración ORDER BY para ordenar los resultados de tu búsqueda. Estamos buscando ordenar ambas columnas en orden ascendente, lo cual se puede especificar usando la palabra clave ASC, sin embargo, también es el orden de clasificación predeterminado, por lo que ASC no es necesariamente requerido.</p>
</details>

In [11]:

%%sql
SELECT *
FROM crime_scene_report


,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his head...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to the knee!,SQL City
5,20180115,theft,Big Bully stole my lunch money!,Chicago
6,20180115,fraud,"Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do\n eiusmod tempor inc...",Seattle
7,20170712,theft,"A lone hunter stalks the night, firing arrows into the Darkness.\n There is no hidin...",SQL City
8,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it is a thing of\n legend, both past and ...",SQL City
9,20171110,robbery,The Gjallarhorn shoulder-mounted rocket system was forged from\n the armor of Guardi...,SQL City


### Entity Relationship Diagram (Optional)

After doing some detective work and finding the tables in the Police Department database, you discover that there is an ERD! This can be really helpful in your quest to find the murderer. If you like, challenge yourself (and your SQL skills) to continue on by just querying the `information_schema` as needed. Otherwise, you can click the dropdown below to reveal the ERD to help you see the tables and relationships in the Police Department database at a glance:

<details>
  <summary>Check the SQL Murder Mystery ERD</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>
</details>

### Diagrama de Relaciones de Entidad (Opcional)

Después de hacer un poco de trabajo de detective y encontrar las tablas en la base de datos del Departamento de Policía, descubres que hay un ERD (Diagrama de Relaciones de Entidad). Esto puede ser realmente útil en tu búsqueda para encontrar al asesino. Si lo prefieres, desafíate a ti mismo (y tus habilidades en SQL) a continuar solo consultando el `information_schema` según sea necesario. De lo contrario, puedes hacer clic en el desplegable a continuación para revelar el ERD que te ayudará a ver las tablas y las relaciones en la base de datos del Departamento de Policía de un vistazo:

<details>
  <summary>Revisa el ERD del SQL Murder Mystery</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>
</details>

In [12]:

%%sql
SELECT *
FROM crime_scene_report
WHERE type='murder' AND date=20180115

,date,type,description,city
0,20180115,murder,Life? Dont talk to me about life.,Albany
1,20180115,murder,"Mama, I killed a man, put a gun against his head...",Reno
2,20180115,murder,Security footage shows that there were 2 witnesses. The first witness lives at the last house o...,SQL City


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('description').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('city').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['date']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'date'}, axis=1)
              .sort_values('date', ascending=True))
  xs = counted['date']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('date', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('description')):
  _plot_series(series, series_name, i)
  fig.legend(title='description', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('date')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['date']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'date'}, axis=1)
              .sort_values('date', ascending=True))
  xs = counted['date']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('date', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('city')):
  _plot_series(series, series_name, i)
  fig.legend(title='city', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('date')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['city'].value_counts()
    for x_label, grp in _df_4.groupby('description')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('description')
_ = plt.ylabel('city')

___

## Resolviendo el Misterio (y verificando si resolviste el caso)

De aquí en adelante, usarás tus habilidades y las pistas en la base de datos del Departamento de Policía para encontrar al asesino. Cuando creas que has encontrado al asesino, vuelve a esta sección y usa la celda de abajo para verificar tu respuesta y ver si encontraste al asesino:

ejemplo:
```python
check_suspect("Adam Levine")
```

In [60]:
check_suspect("Jeremy Bowers")

Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge,try querying the interview transcript of the murderer to find the real villain behind this crime.If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries.
Use this same `check_suspect` function with your new suspect to check your answer.


True

---

## Espacio de Trabajo

Usa las celdas de abajo para escribir tus consultas y trabajar en resolver el misterio. Cuando tengas un sospechoso, verifica tu respuesta usando la [celda arriba de esta](#solving-the-mystery-and-checking-if-you-cracked-the-case) y ejecuta su nombre a través de la función `check_suspect`. Si encuentras al asesino, la función te lo dirá.

No olvides usar `%%sql` en la parte superior de las celdas de `Code` que crees para que tus consultas SQL funcionen. ¡Buena suerte, detectives!

In [19]:
%%sql
SELECT *
FROM crime_scene_report
WHERE type='murder' AND date=20180115

,date,type,description,city
0,20180115,murder,Life? Dont talk to me about life.,Albany
1,20180115,murder,"Mama, I killed a man, put a gun against his head...",Reno
2,20180115,murder,Security footage shows that there were 2 witnesses. The first witness lives at the last house o...,SQL City


Primera busqueda con los datos dados, en la tabla de crimen_scene_report ubicamos  un asesinato ocurrido el dia 15 de enero de 2018 en SQL city.

In [21]:
%%sql
SELECT id, name,address_street_name
FROM person
WHERE name LIKE 'Annabel%' AND address_street_name= 'Franklin Ave'



,id,name,address_street_name
0,16371,Annabel Miller,Franklin Ave


Obtenemos el id de Annabel y asi podremos consular lo que ella vio.

In [23]:
%%sql
SELECT transcript
FROM interview
WHERE person_id=16371

,transcript
0,"I saw the murder happen, and I recognized the killer from my gym when I was working out last we..."


Con el id de Annabel podemos obtener su testimonio del dia, ella vio el asesinato y reconoció al asesino en su gimnasio cuando estaba haciendo ejercicio la semana pasada, el 9 de enero.

In [29]:
%%sql
SELECT *
FROM person
WHERE address_street_name= 'Northwestern Dr'
ORDER BY address_number DESC

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
1,17729,Lasonya Wildey,439686,3824,Northwestern Dr,917817122
2,53890,Sophie Tiberio,957671,3755,Northwestern Dr,442830147
3,73368,Torie Thalmann,773862,3697,Northwestern Dr,341559436
4,96595,Coretta Cubie,303645,3631,Northwestern Dr,378403829
5,19420,Cody Schiel,890431,3524,Northwestern Dr,947110049
6,93509,Emmitt Aceuedo,916706,3491,Northwestern Dr,979073160
7,87456,Leonora Wolfsberger,215868,3483,Northwestern Dr,565203106
8,36378,Freddie Ellzey,267882,3449,Northwestern Dr,474117596
9,53076,Boris Bijou,664914,3327,Northwestern Dr,401191868


Consultando a una persona que viva en Northwestern y ordernardo descendente el numero de la casas, puedo saber quien vive en la ultima casa y voy a revisar con el id si tiene en la tabla interview algo que haya dicho.
0	14887	Morty Schapiro	118009	4919	Northwestern Dr	111564949

In [30]:
%%sql
SELECT transcript
FROM interview
WHERE person_id=14887

,transcript
0,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership nu..."


Con esra consulta de Morty su interview dice:  Tenía una bolsa de "Get Fit Now Gym". El número de membresía en la bolsa comenzaba con "48Z". Sólo los miembros de oro tienen esas bolsas. El hombre se subió a un coche con una placa que decía "H42W".

In [44]:
%%sql
SELECT dri.id, dri.plate_number, per.name, per.address_street_name
FROM drivers_license dri
JOIN person per ON dri.id= per.license_id
WHERE dri.plate_number LIKE 'H42W%'

,id,plate_number,name,address_street_name
0,183779,H42W0X,Maxine Whitely,Fisk Rd


Consultando 2 tablas , usando el JOIN, pude consultar y obtener los datos de la patente H42W y asi obtener el nombre de la persona con su id.

In [49]:
%%sql
SELECT *
FROM get_fit_now_check_in
WHERE membership_id LIKE '48Z%'AND check_in_date=20180109



,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


Consultando la membresia que dijo el testigo con la fecha vista por la otra persona, me arroja 2 personas que son sospechosas.

In [54]:
%%sql
SELECT *
FROM get_fit_now_member get
JOIN get_fit_now_check_in  get_check ON get.id= get_check.membership_id
WHERE get_check.membership_id IN('48Z7A', '48Z55')

,id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,28819,Joe Germuska,20160305,gold,48Z7A,20180109,1600,1730
1,48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700


Aqui puedo ver mas detallado que membresia tiene, para dar con el culpable.

In [58]:
%%sql
SELECT *
FROM facebook_event_checkin face
JOIN person per ON face.person_id= per.id
WHERE per.id IN ('28819','67318')

,person_id,event_id,event_name,date,id,name,license_id,address_number,address_street_name,ssn
0,67318,4719,The Funky Grooves Tour,20180115,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
1,67318,1143,SQL Symphony Concert,20171206,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


En esta consulta me indica que solo Jeremy Bowers tiene evento en su face el dia 15.01.2018, el dia del asesinado. No hay nada de Joe Germuska.

In [59]:
%%sql
SELECT transcript
FROM interview
WHERE person_id=67318

,transcript
0,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" ..."


El es el asesino, pero fue contratado. Dio detalles de quien le pago.

In [62]:
%%sql
SELECT *
FROM facebook_event_checkin
WHERE event_name= 'SQL Symphony Concert' And date BETWEEN 20171201 AND 20171231
ORDER BY person_id ASC


,person_id,event_id,event_name,date
0,11173,1143,SQL Symphony Concert,20171223
1,19260,1143,SQL Symphony Concert,20171214
2,19292,1143,SQL Symphony Concert,20171213
3,24397,1143,SQL Symphony Concert,20171208
4,24556,1143,SQL Symphony Concert,20171207
5,24556,1143,SQL Symphony Concert,20171221
6,24556,1143,SQL Symphony Concert,20171224
7,28582,1143,SQL Symphony Concert,20171220
8,28582,1143,SQL Symphony Concert,20171215
9,43366,1143,SQL Symphony Concert,20171207


Aqui tengo a 2 personas que asistieron 3 veces en diciembre a ese concierto. Tengo el person_id para seguir indagando.

In [63]:
%%sql
SELECT *
FROM person
WHERE id IN( '24556' , '99716')


,id,name,license_id,address_number,address_street_name,ssn
0,24556,Bryan Pardo,101191,703,Machine Ln,816663882
1,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


EL asesino dijo que era una mujer y la unica mujer es Miranda Priestly

In [68]:
%%sql
SELECT id, hair_color,height, plate_number,car_make, car_model
FROM drivers_license
WHERE id= '202298'

,id,hair_color,height,plate_number,car_make,car_model
0,202298,red,66,500123,Tesla,Model S


Aqui se puede decir que ella tiene la altura informada y el carro que dijo el asesino.

In [76]:
%%sql
SELECT *
FROM get_fit_now_member get
JOIN get_fit_now_check_in get_check ON get.id= get_check.membership_id
WHERE get.id= '202298'

,id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time


In [78]:
%%sql
SELECT *
FROM income
WHERE ssn=987756388

,ssn,annual_income
0,987756388,310000


No hay nada